In [8]:
from dcicutils import ff_utils
from dcicutils import s3Utils
# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

my_env = 'data'
my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)

my_s3_util = s3Utils(env=my_env)
raw_bucket = my_s3_util.raw_file_bucket
out_bucket = my_s3_util.outfile_bucket

file_not_found = 0
uploading = 0
success = 0
patch_problem = 0

file_url = '/search/?type=File&file_size=No+value&limit=all'+ \
           '&status=uploaded&status=released+to+project&status=released'
files_resp = ff_utils.search_metadata(file_url, key = my_auth)
print len(files_resp)

no_size = 0
counter = 0
for a_file in files_resp:
    counter += 1
    if counter % 100 == 0:
        print counter
    if a_file['status'] in ['uploading', 'to be uploaded by workflow']:
        uploading += 1
        continue
    # check if there is  no filesize
    if not a_file.get('file_size'):
        # decide on the bucket
        if 'FileProcessed' in a_file['@type']:
            bucket = out_bucket
        else:
            bucket = raw_bucket
            
        # check if file is in s3
        head_info = my_s3_util.does_key_exist(a_file['upload_key'], bucket)
        if not head_info:
            print a_file['@id'], a_file['status']
            file_not_found += 1
            continue
        file_size = head_info['ContentLength']

        patch_data = {'file_size': file_size}
        try:
            ff_utils.patch_metadata(patch_data, obj_id=a_file['uuid'] ,key=my_auth)
            success += 1
        except Exception as e:
            print e
            print
            patch_problem += 1

print file_not_found, 'files missing from s3'
print patch_problem, 'files had patch problems'
print uploading, 'uploading files skipped'
print success, 'total files patched'

0
0 files missing from s3
0 files had patch problems
0 uploading files skipped
0 total files patched
